# IT Security - Sheet 2 "Symmetric Encryption"

**Total achievable points: 20**

**Released: 31.10.2023**

**Submission Deadline: 07.11.2023 23:59**

---
Groupnumber: 128

Names and matriculation numbers of **ALL** team members: Samuel Rode (445160), Nils Maasch (445796), Pau Azpeita Bergos (443428), Gereon Geuchen (445328), Ben-Jay Huckebrink (445219) 


Format: John Doe (999999)

---

**Important Information**

The assignments have to be submitted by groups of 5 students. Even if you are registered in RWTHmoodle to a submission group, **please include the group number as well as the name and matriculation number of every group member in this notebook**. In case you are not part of a submission group and want to hand in assignments, please contact `ba-itsec@itsec.rwth-aachen.de`. We will then assign you to a submission group.

Enter your solutions for the tasks in the respective cells of this notebook. These cells are either marked by "YOUR ANSWER HERE" or `#YOUR CODE HERE`. Please do not add any new cells or remove existing ones, especially do not copy cells. Cells marked with `###PLAYGROUND` can be used to test your implementation and generate output (see example for the first tasks). Please do not add any other output or tests in the cell of the task, just implement the function with the header provided. If you want to test your implementation, use the `###PLAYGROUND` cells. They will be ignored during grading. **Do not change any other cells or add new ones.**

Please **do not import any further Python packages** except the default Python ones and the ones that are explicitly given by us.

## Content of this Assignment

In the lecture, you learned about symmetric cipher standards such as DES and AES. It was shown, how to deal with the block ciphers in different modes of encryption. Also, you learned about hash functions, MDCs, and MACs. 
In this exercise, you'll take a deeper look into one of the modes presented in the lecture and be introduced to a new mode, the Output-Feedback-Mode. Also, you can again try to break some stuff (not the autograding of the assignments :D) by exploiting a bad MAC construction! But before, let's take a little detour to *One Time Pad* (OTP).

## 1. One-Time Pad (4 Points)

In the lecture, the concept of the One-Time-Pad Cipher was introduced. This cipher is also called *Vernam Cipher* or *Vernam's One-Time-Pad*. This cipher has a few advantages and disadvantages that make using this cipher a bit cumbersome.

The following tasks will provide some useful thoughts about the properties of One-Time-Pad. This task is a bit different from the other programming tasks, as it should make you practically think about the properties of OTP. So sometimes, in this task the answer to a programming question can be very simple. 

We will implement the functions necessary to encrypt and decrypt using OTP. The given functions `to_binary(message: str) -> str` and `from_binary(binray_message) -> str` will translate a given string into or from its binary representation. This representation is also a string, containing just `0` and `1`.

In [1]:
def to_binary(message: str) -> str:
    binary_message = ''.join(format(ord(char), '08b') for char in message)
    return binary_message

def from_binary(binary_message: str) -> str:
    message = ''.join(chr(int(binary_message[i:i+8], 2)) for i in range(0, len(binary_message), 8))
    return message

### Task 1.1 (0.5 Point)

Implement the function `xor_binary_messages(first_message: str, second_message: str) -> str`. This function will get two strings of just `0` and `1` and will produce the xor of them. The output should also be a string, just containing `0` and `1`.

In [2]:
def xor_binary_messages(first_message: str, second_message: str) -> str:
    # YOUR CODE HERE
    xor_binary_message = ''.join('0' if tup[0] == tup[1] else '1' for tup in zip(first_message, second_message))
    xor_binary_message += "1"*abs(len(first_message)-len(second_message))
    return xor_binary_message

In [3]:
### PLAYGROUND
# You can use this cell to test out your implementation. Everything in this cell will be ignored during grading.

test1_string = '00001111'
test2_string = '10010110'

print(xor_binary_messages(test1_string, test2_string))

10011001


In [4]:
# This test just checks the output format of your solution

test1_string = '00001111'
test2_string = '10010110'

result = xor_binary_messages(test1_string, test2_string)

assert all(char in '01' for char in result)

In [ ]:
# Even this cell seems empty, it contains automatic tests. Please do not remove this cell and just ignore it.

### Task 1.2 (0.5 Points)

Now, as we have everything we need, implement the following two functions. `encrypt(message: str, key: str) -> str` takes a message and a key. The key is given in a binary string, so a string with just `0` and `1`. The function returns the binary representation of the encryption, also a binary string. The function `decrypt(cipher: str, key: str) -> str` reverses this process and takes a cipher in the form of a binary string, as well as the key again. This function will return a string containing the original message. This string has to be human-readable, so not just the binary representation.

In [5]:
def encrypt(message: str, key: str) -> str:
    # YOUR CODE HERE
    return xor_binary_messages(to_binary(message), key)

def decrypt(cipher: str, key: str) -> str:
    # YOUR CODE HERE
    return from_binary(xor_binary_messages(cipher, key))

In [6]:
### PLAYGROUND
# You can use this cell to test out your implementation. Everything in this cell will be ignored during grading.
plain = 'Hello World!'
key = '010010001110011101100011101111110011010101100000101000010101111000001011101001011111100100111001'
cipher = '000000001000001000001111110100110101101001000000111101100011000101111001110010011001110100011000'

encr = encrypt(plain, key)
decr = decrypt(cipher, key)

print(encr)
print(decr)
print(decrypt(encr, key))

000000001000001000001111110100110101101001000000111101100011000101111001110010011001110100011000
Hello World!
Hello World!


In [7]:
# This test just checks the output format of your solution

plain = 'Hello World!'
key = '010010001110011101100011101111110011010101100000101000010101111000001011101001011111100100111001'
cipher = '000000001000001000001111110100110101101001000000111101100011000101111001110010011001110100011000'

encr = encrypt(plain, key)
decr = decrypt(cipher, key)

assert all(char in '01' for char in encr)
assert not all(char in '01' for char in decr)

In [ ]:
# Even this cell seems empty, it contains automatic tests. Please do not remove this cell and just ignore it.

### Task 1.3 - The Espionage Thriller (3 Points)

Alice is a spy on a mission. At the beginning of her mission, she got a random bit string. She can use this to encrypt messages using OTP to send them to Bob in the operations central. Eve is a spy from a counterintelligence agency and tries to intercept messages and to mess around as much as possible.

Now, Alice wants to send a very important message to Bob and uses OTP for that. This important message just contains a date, which is stored in `true_date_plain`. Alice used the provided key to encrypt the message.

Implement the function `return_encrypted_date() -> str` that just returns the given `true_date_plain` OTP encrypted using the key `secret_key`.

In [8]:
secret_key = '00000001101100110110101110010000101001011000000100101100100010000101111011011100'
true_date_plain = '24.12.2024'

def return_encrypted_date() -> str:
    # YOUR CODE HERE
    return encrypt(true_date_plain, secret_key)

Bob received the encrypted message from Alice and he has access to the same key that Alice used. Bob decrypts the message.

Implement the function `return_decrypted_date() -> str` to check if Bob is able to see the correct date. The function should just return the decrypted message. The received message is given in `received_message` and the same key `secret_key` is used.

In [9]:
received_message = return_encrypted_date()

def return_decrypted_date() -> str:
    # YOUR CODE HERE
    return decrypt(received_message, secret_key)

In [10]:
### PLAYGROUND
# You can use this cell to test out your implementation. Everything in this cell will be ignored during grading.

print(return_encrypted_date())
print(return_decrypted_date())

00110011100001110100010110100001100101111010111100011110101110000110110011101000
24.12.2024


In [ ]:
# Even this cell seems empty, it contains automatic tests. Please do not remove this cell and just ignore it.

Eve was able to intercept the message to Bob. She knew it was a date and she is very confident that the message was `12.12.2024` as this marks the final date of Eve's mission. She then tries to compute the key. 

Use your knowledge from the lecture about OTP. Think about how Eve can reconstruct a key using the given information. Provide the key Eve reconstructed as the return of the function `return_reconstructed_key() -> str`. **You can either write code that computes it or just return the answer.**

In [11]:
def return_reconstructed_key() -> str:
    # YOUR CODE HERE
    return xor_binary_messages(to_binary('12.12.2024'), received_message)

In [12]:
### PLAYGROUND
# You can use this cell to test out your implementation. Everything in this cell will be ignored during grading.

print(return_reconstructed_key())

00000010101101010110101110010000101001011000000100101100100010000101111011011100


In [ ]:
# Even this cell seems empty, it contains automatic tests. Please do not remove this cell and just ignore it.

Eve is not sure if the reconstructed key is correct and she was unable to decrypt any other messages from Alice to Bob in a way they would make any sense. However, she wants to make sure that Bob does not get correct dates from Alice anymore. She knows that Alice is just sending dates and these dates are not the first or last day of a month, because all spies wolrdwide are off on these days. Eve wants to modify the messages so that the date is moved by one day in any direction. Eve is able to intercept and modify the messages before they will arrive at Bob.

Implement a function that modifies an encrypted date in the explained way and returns the modified message as return to the function `modify(message:str) -> str`. The argument `message` will be an encrypted date.

In [13]:
def modify(message: str) -> str:
    # YOUR CODE HERE
    replacement = "1" if message[15] == '0' else "0"
    return message[:15] + replacement + message[16:]

In [14]:
### PLAYGROUND
# You can use this cell to test out your implementation. Everything in this cell will be ignored during grading.
message = "00000010101101000110101110010000101001011000000100101100100010000101111011011100"
print(message)
test = modify(message)
print(test)

date = "19.12.2024"
encrypted_date = encrypt(date, secret_key)

print(f"Orig. date: {decrypt(encrypted_date, secret_key)}") # For sanity checking
print(f"Mod. date:  {decrypt(modify(encrypted_date), secret_key)}")

00000010101101000110101110010000101001011000000100101100100010000101111011011100
00000010101101010110101110010000101001011000000100101100100010000101111011011100
Orig. date: 19.12.2024
Mod. date:  18.12.2024


In [ ]:
# Even this cell seems empty, it contains automatic tests. Please do not remove this cell and just ignore it.

Eve was able to intercept even more messages from Alice. She successfully modified them and forwarded them to Bob. However, Eve is confident that Alice has a limited OTP key only, so she suspects that Alice is reusing it for every new message. Eve also knows that the communicated dates are in December 2024 and one of the intercepted dates is either the 18.12.2024 or the 04.12.2024. Eve knows how OTP works and also knows the weakness of using the same key again. In the following we want to decrypt the messages even without knowledge of the key.

Implement a function to reconstruct the messages Alice sent. There are two messages given and both contain a date. The dates are both in December 2024. Under the assumption that for the encryption the same key was used, exploit the XOR construction of OTP to reconstruct the messages.

Implement the function `get_both_dates(date1: str, date2) -> (str, str)` that returns both dates. The arguments and variables `date1` and `date2` are the encrypted dates. The order of them is not important. Use the given function `check_for_date(date: str) -> bool` to check for a valid date automatically.

In [15]:
date1 = '10100111100011100010000101110010010000100001110001101000010010010110111011110000'
date2 = '10100100100000000010000101110010010000100001110001101000010010010110111011110000'

import re

def check_for_date(date: str) -> bool:
    return all(char in '0123456789.' for char in date) and re.match(r"^(0[1-9]|[12][0-9]|3[01])\.(0[1-9]|1[0-2])\.(\d{4})$", date)

In [16]:
def get_both_dates(date1: str, date2: str) -> (str, str):
    # YOUR CODE HERE
    known_dates = ["04.12.2024", "18.12.2024"]
    for known_date in known_dates:
        
        known_date_binary = to_binary(known_date)
        
        possible_key1 = xor_binary_messages(known_date_binary, date1)
        possible_key2 = xor_binary_messages(known_date_binary, date2)
        
        decrypted_date2 = decrypt(date2, possible_key1)
        if check_for_date(decrypted_date2):
            decrypted_date1 = decrypt(date1, possible_key1)
            if check_for_date(decrypted_date1):
                return (decrypted_date1, decrypted_date2)
        
        decrypted_date1 = decrypt(date1, possible_key2)
        if check_for_date(decrypted_date1):
            decrypted_date2 = decrypt(date2, possible_key2)
            if check_for_date(decrypted_date2):
                return (decrypted_date1, decrypted_date2)
    
    return None, None


In [17]:
### PLAYGROUND
# You can use this cell to test out your implementation. Everything in this cell will be ignored during grading.
print(get_both_dates(date1, date2))

('18.12.2024', '26.12.2024')


In [ ]:
# Even this cell seems empty, it contains automatic tests. Please do not remove this cell and just ignore it.

Alice already supected that someone was intercepting their messages and used two different keys to encrypt message `date1` and `date2`. The keys are given below as `key1` and `key2`. Bob was able to decrypt the messages using these two keys. What was in there?

Implement the function `bob_decrypt(message1: str, message2: str, key1: str, key2: str) -> str` to print and return a string containing both messages concatenated (order: `date1|date2`, decrypt `date1` with `key1` and `date2` with `key2`).

In [18]:
key1 = '11101110110110100111001000010111001000010011110000000001001110100100111010000110'
key2 = '11000001111100100101100001010010001000010111001100000111001001010100111111010001'

def bob_decrypt(message1: str, message2: str, key1: str, key2: str) -> str:
    # YOUR CODE HERE
    return decrypt(message1, key1) + decrypt(message2,key2)

In [ ]:
# Even this cell seems empty, it contains automatic tests. Please do not remove this cell and just ignore it.

04.12.202418.12.2024


## 2. Modes of Encryption (9.5 Points)

We know from the lecture that we use modes of encryption to encrypt messages that are longer than a singular block (e.g. 128 bit). We will now explore a few properties of these modes.

### Task 2.1 (1.5 Points)

From the lecture you already know the three modes of encryption *Electronic Codebook Mode* (ECB), *Cipher Block Chaining Mode* (CBC), and *Counter Mode* (CTR). **Describe** the main problem of the *Electronic Codebook Mode*, what an attacker can learn about an ECB-encrypted message, and how this is compensated using *Cipher Block Chaining Mode* or *Counter Mode*.

The main problem is that the same blocks of plaintext are encrypted as the same blocks of ciphertext. Thus, patterns are still recognizable, as patterns in the plaintext directly translate to patterns in the cipher text.\
Cipher Block Chaining uses the ciphertext of the last block to encrypt the next block, thus, repetitions are avoided.

### Task 2.2 (1 Points)

**Explain** why the *Cipher Block Chaining Mode* and the *Counter Mode* need an *Initialization Vector* (IV). Additionally, **name and describe** problems that can occur by selecting the IV.

An IV is needed in order to be able to encrypt the very first block, since every encryption depends on the encryption of the block's predecessor and the first block does not have a predecessor. When reusing the IV, the same starting blocks will always be encrpyted in the same way, thus revealing whether they are the same. Such modes are vulnerable to padding oracle attacks.

### Task 2.3 (3 Points)

One mode of encryption you should already know about from the lecture is *Cipher Block Chaining* (CBC). In CBC, we use the encrypted output of the cipher from the previous block to chain it into the encryption of the next plaintext block. In the following, you can show that you are an expert in encryption modes by implementing a (simplified) CBC mechanism.

Implement the functions `CBC_encrypt(message, iv, k)` and `CBC_decrypt(ciphertext, iv, k)` which encrypt/decrypt a given text according to an initialization vector `iv` and a key `k` in CBC mode. They should both return a string with the encryption/decryption result. Ensure that your functions only accept texts that are a multiple of 8 and throw a `ValueError` otherwise (we disregard anything about padding here).

Use the following encryption/decryption functions to encrypt/decrypt a single block of length `l=8`.

Hint: your `iv` and `k` need to have the same length as a singular encrypted block.

In [19]:
def encrypt_block(message: str, key: str) -> str:
    if len(message) != 8:
        raise ValueError("Message length must be 8!")
    if len(key) != 8:
        raise ValueError("Key length must be 8!")
    
    encrypted_message = []
    for i in range(len(message)):
        encrypted_char = chr(ord(message[i]) ^ ord(key[i % len(key)]))
        encrypted_message.append(encrypted_char)
    return ''.join(encrypted_message)

In [20]:
def decrypt_block(ciphertext: str, key: str) -> str:
    if len(ciphertext) != 8:
        raise ValueError("Ciphertext length must be 8!")
    if len(key) != 8:
        raise ValueError("Key length must be 8!")
    
    decrypted_message = []
    for i in range(len(ciphertext)):
        decrypted_char = chr(ord(ciphertext[i]) ^ ord(key[i % len(key)]))
        decrypted_message.append(decrypted_char)
    return ''.join(decrypted_message)

In [ ]:
def CBC_encrypt(message: str, iv: str, k: str) -> str:
    # YOUR CODE HERE
    if (len(message) % 8) != 0:
        raise ValueError("Plaintext length must be a multiple of 8!") 
    if len(message) == 0:
        return ""

    plaintext_blocks: list[str] = [message[i:i+8] for i in range(0, len(message), 8)]
    ciphertext_blocks: list[str] = []

    for (idx, block) in enumerate(plaintext_blocks):
        if idx == 0:
            xor_res = "".join(chr(ord(block[i]) ^ ord(iv[i])) for i in range(0, 8))
            ciphertext_blocks.append(encrypt_block(xor_res, k))
        else:
            xor_res = "".join(chr(ord(block[i]) ^ ord(ciphertext_blocks[idx - 1][i]))\
                              for i in range(0, 8))
            ciphertext_blocks.append(encrypt_block(xor_res, k))

    return "".join(ciphertext_blocks)

In [ ]:
def CBC_decrypt(ciphertext: str, iv: str, k: str):
    # YOUR CODE HERE
    if (len(ciphertext) % 8) != 0:
        raise ValueError("Ciphertext length must be a multiple of 8!")
    if len(ciphertext) == 0:
        return ""

    ciphertext_blocks: list[str] = [ciphertext[i:i+8] for i in range(0, len(ciphertext), 8)]
    plaintext_blocks: list[str] = []

    for (idx, block) in enumerate(ciphertext_blocks):
        if idx == 0:
            decrypt_res = decrypt_block(block, k)
            plaintext_blocks.append("".join(chr(ord(decrypt_res[i]) ^ ord(iv[i])) for i in range(0, 8)))
        else:
            decrypt_res = decrypt_block(block, k)
            plaintext_blocks.append("".join(chr(ord(decrypt_res[i]) ^ ord(ciphertext_blocks[idx - 1][i]))\
                                            for i in range(0, 8)))
    
    return "".join(plaintext_blocks)

In [ ]:
# Even this cell seems empty, it contains automatic tests. Please do not remove this cell and just ignore it.

### Task 2.4 (4 points)

We now turn to another mode that you should know about: *Output Feedback Mode* (OFB). The main idea here is that we can generate a keystream through encrypting the initialization vector again and again. Then, we only need to XOR the plaintext with the keystream. This means, we can even pre-generate such a keystream!

For this task, you can again use the functions `encrypt_block` and `decrypt_block` as given above.

However, this time, you have to implement the functions `OFB_encrypt(message, iv, k)` and `OFB_decrypt(message, iv, k)` which encrypt/decrypt a given text according to an initialization vector `iv` and a key `k` in the simplified Output Feedback (OFB) mode. 

They should both again return a string with the encryption/decryption result. Also, ensure that your functions only accept texts that are a multiple of 8 and throw a `ValueError` otherwise (we disregard anything about padding again).

Encryption and decryption in OFB work as follows:

$C_i = E^i(IV, k) \oplus P_i$

$P_i = C_i \oplus E^i(IV, k)$

where $E^i(IV, k)$ means that the $IV$ is encrypted $i$ times with the key $k$.

as also shown in the image below. 

![](./ofb.png)

In [30]:
def OFB_encrypt(message: str, iv: str, k: str) -> str:
    # YOUR CODE HERE
    if (len(message) % 8) != 0:
        raise ValueError("Plaintext length must be a multiple of 8!")
    if len(message) == 0:
        return ""

    plaintext_blocks: list[str] = [message[i:i+8] for i in range(0, len(message), 8)]
    ciphertext_blocks: list[str] = []

    iv_encrypted = encrypt_block(iv, k)
    for block in plaintext_blocks:
        xor_res = "".join(chr(ord(block[i]) ^ ord(iv_encrypted[i]))\
                          for i in range(0, 8))     
        ciphertext_blocks.append(xor_res)
        iv_encrypted = encrypt_block(iv_encrypted, k)

    return "".join(ciphertext_blocks)

In [32]:
def OFB_decrypt(ciphertext: str, iv: str, k: str) -> str:
    # YOUR CODE HERE
    # Use self-inverse nature of OFB
    return OFB_encrypt(ciphertext, iv, k)

In [ ]:
# Even this cell seems empty, it contains automatic tests. Please do not remove this cell and just ignore it.

## 3. Length Extension Attack (5.5 Points)

Now, we turn to the topic of integrity protection. You learned about Message Authentication Codes (MACs) that can be constructed from cryptographic hash functions or block ciphers. We do exactly that and use a secure block cipher in CBC mode in the following.

For this task, we assume a secure block cipher in CBC mode is used to calculate a MAC for a message `m`. The last cipher text block denotes the corresponding MAC here. 

For example, consider a message with four 128-bit blocks. The last cipher text block ```c4``` of `c=c1||c2||c3||c4` is the MAC for the message, ```m = m1||m2||m3||m4``` where `c` is the corresponding ciphertext to `m`. The initialization vector ```iv``` only consists of zero bits.

However, this simple construction is vulnerable to a so-called length extension attack. You are given two message and MAC pairs ```(m1, t1), (m2, t2)```, and your task is to forge a valid message-MAC pair without knowing the secret key. 

One secret key for the exercise is given, so that you can test your attack string. Note, that another key and different message-MAC pairs are used in the grading cells.

For this exercise, we use the library [Cryptography](https://cryptography.io/en/latest/) that provides many cryptographic algorithms. You have to use AES128 as the block cipher here. To work with the algorithms, you need to use byte-encoded strings, as shown below.

In [26]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes

message_1 = b"This is my message string and needs to be MACed."
mac_1 = b'&\x92\x07x_Z\xa9.\xff?\xd8\xc5\xbbM\xc9\xaa'

message_2 = b"This is another message and also needs a MAC!!!!"
mac_2 = b'\x9c5\x93\x9b\xfe\xd0\x87\x86\xeb\x80b\xfa\x16`\xaad'

secret_key = b'\tmc\xfc\x89\xc8\x05\x03k\xf81\x93\xa3]\x021'

### Task 3.1 (1.5 points)

Implement a function ```create_MAC(text, key)``` which returns the computed CBC-MAC of the `message` with AES128 as the used block cipher.

Hint: CBC-MAC works by setting the `IV` to all zeros.

In [27]:
def create_MAC(message, key) -> str:
    iv = bytes(16)
    cipher = Cipher(algorithms.AES128(key), modes.CBC(iv))
    encryptor = cipher.encryptor()

    encrypted_msg = encryptor.update(message) + encryptor.finalize()
    return encrypted_msg[-16:]

In [ ]:
# Even this cell seems empty, it contains automatic tests. Please do not remove this cell and just ignore it.

### Task 3.2 (2 points)

You now need to send an authenticated message `forge_message` to a server, such that the MAC of `forge_message` is equal to the MAC of `m2`. 

Take a closer look at the CBC-mode to construct such a message. Ensure that `forge_message` is different to `m2`.

Implement a function `construct_forge(m1, mac1, m2, mac2)` that returns such a message together with the corresponding MAC as a tuple `(forge_message, mac)`.

In [28]:
def construct_forge(m1, mac1, m2, mac2):
    # As per slide 28 of ch. 3, we have that if we have (m1, mac1) & (m2, mac2), we can construct
    # the _valid_ pair m1 || (m2 XOR mac1) (with the MAC being mac2)
    # However, as m1 & m2 are not guaranteed to be _one-block_ messages, we do not _fully_ XOR m2 & mac1,
    # but rather _only the first block_ as to "eradicate" the MAC of m1 in the following XOR calculations,
    # meaning we still end up with the MAC being mac2, as desired
    m1_array = list(m1)

    # Why does the `bytes` class not have an overloaded XOR operator?
    m2_first_block = list(m2)[:16]
    m2_first_xored = [a ^ b for (a, b) in zip(m2_first_block, mac1)]
    m2_suffix = list(m2)[16:]

    m1_array.extend(m2_first_xored)
    m1_array.extend(m2_suffix)

    return (bytes(m1_array), mac2)

In [ ]:
# Even this cell seems empty, it contains automatic tests. Please do not remove this cell and just ignore it.

### Task 3.3 (2 points)

Now you have to implement a test function for the 'server side'. The server should accept a `message` if the provided `mac` is correct with a given secret `key`. Implement a function `server_auth(message, t, key)` which returns `True` or `False` depending on the check.

In [29]:
def server_test_MAC(message, mac, k):
    calculated_mac = create_MAC(message, k)
    return (mac == calculated_mac)

In [ ]:
# Even this cell seems empty, it contains automatic tests. Please do not remove this cell and just ignore it.

## 4. Example Exam Task (0.5 Points)

### Task 4.1 - Symmetric Encryption

**Name** one advantage of the *Counter Mode* (CTR) over the *Cipher Block Chaining Mode* (CBC) or *Output Feedback Mode* (OFB).

CTR mode encryption and decription can be parallelized

### Task 4.2 - OTP

**Prove** that One Time Pad is perfectly secure.

Since |P| = |C| = |K|, we can use Shannon's Therem. Let P, C be a pair of plain- and ciphertext, chosen uniformly at random. Then K := C xor P is the key such that E_K(P) = C. Since K is chosen uniformly at random, OTP provides perfect secrecy according to Shannon's Theorem.

### Task 4.3 - Integrity Protection

**Name** the differences and similarities of a *Modification Detection Code* (MDC) and a *Message Authentication Code* (MAC).

Similarities: Both try to detect Data Modification but unauthorized entities. Both might use hash functions to compute a Tag/Fingerprint.
Differences: MDC requires a secure channel, while in MAC, the tag is part of the message send through the insecure channel.

## 5. Feedback (0.5 points)

You made it through another assignment! Since we want to know how it went and how we might improve the exercises, we include the following task. Here, you can write constructive feedback! You even get 0.5 points for it if you write anything. But don't worry, we do not grade the content itself!

Grading hint: it is enough if there is just anything

anything :D\
On a more serious note: 
The "more theoretical" approach of exercise 1 was a nice change of pace (in comparison to the other coding exercises).

For exercises 2.3 & 2.4 -- as seen with the questions in RWTHmoodle -- it was a bit confusing that the messages, keys etc. given here are _not_ these "binary strings" from exercise 1, but rather are to be treated "properly" like in the functions `encrypt_block`/`decrypt_block` (with ASCII code conversion, etc.); we would ask for such things to be made clear explicitly in the future.

For exercise 3, whilst it is good that a link to the documentation of the cryptography library was provided, it was a bit of a hassle to work with/find out how to work with the `bytes` type in Python; therefore, in the future, it would be appreciated if you could either provide a good documentation for that type (& _no_, the official Python documentation on data types is _not_ good in that regard, as it is just a mere wall of text with all possible functions listed & no real context given) or give examples of the necessary functions in regards to that `bytes` type in the exercise notebook itself.